# Dask Array (1)

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Dask"
* https://docs.dask.org/en/latest/array.html
* https://docs.dask.org/en/stable/array-chunks.html
* https://en.wikipedia.org/wiki/Taxicab_geometry
* https://docs.h5py.org/en/stable/

## Задачи для совместного разбора

In [2]:
dask.__version__

'2023.3.2'

In [3]:
#!pip install --user --upgrade dask

In [1]:
import dask.array as da
import numpy as np

1. Создайте массив размерностью 1000 на 300000, заполненный числами из стандартного нормального распределения. Исследуйте основные характеристики полученного массива.

In [6]:
%%time
arr_np = np.random.standard_normal(size=(1000, 300_000))

CPU times: total: 8.42 s
Wall time: 23.1 s


In [7]:
%%time
arr_da = da.random.standard_normal(size=(1000, 300_000))
print('done')
arr_da

done
CPU times: total: 0 ns
Wall time: 7.48 ms


dask.array<standard_normal, shape=(1000, 300000), dtype=float64, chunksize=(1000, 16777), chunktype=numpy.ndarray>

In [10]:
%%time
m = arr_da.mean(axis=0)
m.compute()

CPU times: total: 26 s
Wall time: 4.99 s


array([ 0.00880982, -0.02663483, -0.02281334, ..., -0.02254877,
       -0.0267781 ,  0.01677565])

2. Посчитайте сумму квадратов элементов массива, созданного в задаче 1. Создайте массив `np.array` такого же размера и сравните скорость решения задачи с использование `da.array` и `np.array`

In [11]:
%%time
arr_np = np.random.standard_normal(size=(1000, 300_000))
(arr_np ** 2).sum()

CPU times: total: 10.2 s
Wall time: 24.7 s


299995611.3999972

In [12]:
%%time
arr_da = da.random.standard_normal(size=(1000, 300_000))
(arr_da ** 2).sum().compute()

CPU times: total: 28.6 s
Wall time: 5.42 s


300003192.05170363

In [13]:
#(arr_da ** 2).sum().visualize()

In [14]:
arr_da.to_hdf5("demo.h5", "/x")

In [15]:
import h5py

In [19]:
%%time
with h5py.File("demo.h5") as fp:
    print(fp)
    print(fp.keys())
    dset = fp["x"]
    print(dset)
    arr1 = dset[:, :150_000]

<HDF5 file "demo.h5" (mode r)>
<KeysViewHDF5 ['x']>
<HDF5 dataset "x": shape (1000, 300000), type "<f8">
CPU times: total: 297 ms
Wall time: 1.36 s


In [20]:
arr1.shape

(1000, 150000)

In [ ]:
# fp = h5py.File("demo.h5")
# dset = fp["x"]
# arr1 = dset[:, :150_000]
# fp.close()

In [23]:
%%time
s = arr_da.sum()
print(s.compute())
m = arr_da.mean()
print(m.compute())

-8151.660028720719
-2.717220009573573e-05
CPU times: total: 53.2 s
Wall time: 10 s


In [24]:
%%time
s = arr_da.sum()
m = arr_da.mean()
dask.compute(s, m)

CPU times: total: 28 s
Wall time: 5.38 s


(-8151.660028720719, -2.717220009573573e-05)

## Лабораторная работа 7

__При решении данных задач не подразумевается использования циклов или генераторов Python в ходе работы с пакетами `numpy`, `pandas` и `dask`, если в задании не сказано обратного. Решения задач, в которых для обработки массивов `numpy`, структур `pandas` или структур `dask` используются явные циклы (без согласования с преподавателем), могут быть признаны некорректными и не засчитаны.__

В ходе выполнения все операции вычислений (расчет средних значений, расчет косинусной близости и т.д.) проводятся над `dask.array` и средствами пакета `dask`, если в задании не сказано обратного. Переход от `dask.array` к `numpy.array` или `pd.DataFrame` возможен исключительно для демонстрации результата в конце решения задачи. Если в задаче используются результаты выполнения предыдущих задач, то подразумевается, что вы используете результаты в виде `dask.array` (то есть то, что было получено до вызова `compute`, а не после).

In [1]:
import dask.array as da

<p class="task" id="1"></p>

1\. Считайте датасет `embeddings` из файла `recipe_embeddings.h5` в виде `dask.array`. Выведите на экран основную информацию о массиве: размер, количество векторов $M$ (количество строк в массиве), размерность каждого вектора $N$ (количество столбцов в массиве), тип, количество и размер сегментов. 

In [2]:
import h5py

In [3]:
fp = h5py.File("recipe_embeddings.h5")
embeddings = fp["embeddings"]
emb = da.array(embeddings)
emb

dask.array<array, shape=(1200000, 312), dtype=float32, chunksize=(107546, 312), chunktype=numpy.ndarray>

In [159]:
# количество векторов, размерность каждого из них
emb.shape

(1200000, 312)

In [62]:
fp.close()

<p class="task" id="2"></p>

2\. Посчитайте и выведите на экран среднее значение всех элементов массива. Исследуйте, как влияет значение аргумента `chunks` при создании `dask.array` на скорость выполнения операции поиска среднего. 

Пусть $M$ - количество строк в массиве, $N$ - количество столбцов в массиве, `chunks=(r,c)`. Сравните несколько вариантов:
* $r=M$, $с \ll N$ , 
* $r \ll M$, $c=N$ 
* $r = M$, $c = N$ 
* значения $r, c$ по умолчанию.

Выберите наиболее оптимальные значения $r$ и  $c$ в смысле скорости вычислений и далее продолжайте работу с ними.

In [5]:
M, N = emb.shape[0], emb.shape[1]

In [37]:
%%time
r = M
c = N / 2
emb1 = da.asarray(emb, chunks=(r, c))
emb1.mean().compute()

CPU times: total: 1.48 s
Wall time: 1.5 s


0.0023777557

In [40]:
%%time
r = M / 2
c = N 
emb2 = da.asarray(emb, chunks=(r, c))
emb2.mean().compute()

CPU times: total: 1.12 s
Wall time: 1.5 s


0.0023777557

In [41]:
%%time
r = M
c = N 
emb3 = da.asarray(emb, chunks=(r, c))
emb3.mean().compute()

CPU times: total: 953 ms
Wall time: 1.47 s


0.0023777557

<p class="task" id="3"></p>

3\. Опишите пространство, в котором расположены эмбеддинги, посчитав минимальное и максимальное значение для каждой из координат. Сведите результаты в таблицу `pd.DataFrame`, состоящую из двух строк и 312 столбцов. Задайте индексы строк "min" и "max". Названия столбцов сделайте вида $e_i$. Выведите полученную таблицу на экран.

Решите задачу двумя способами. В первом варианте сделайте два вызова метода `compute` для расчета каждого из векторов максимальных и минимальных значений. Во втором варианте сделайте один вызов функции `dask.compute` для одновременного расчета двух векторов. Сравните время выполнения двух решений.

In [61]:
import pandas as pd

In [90]:
es = [f'e{i}' for i in range(1, 313)]

In [91]:
%%time
maxs = emb3.max(axis=0).compute()
mins = emb3.min(axis=0).compute()
pd.DataFrame([mins, maxs], index=['min', 'max'], columns=es)

CPU times: total: 4.67 s
Wall time: 3.2 s


,e1,e2,e3,e4,e5,e6,e7,e8,e9,e10,...,e303,e304,e305,e306,e307,e308,e309,e310,e311,e312
min,-0.132803,-0.149056,-0.094468,-0.191697,-0.114229,-0.114341,-0.096039,-0.115178,-0.157275,-0.116715,...,-0.103254,-0.122285,-0.149789,-0.127703,-0.094802,-0.119690,-0.141425,-0.123732,-0.081543,-0.227348
max,0.135038,0.076125,0.157854,0.030987,0.101192,0.111774,0.147497,0.173821,0.099808,0.115573,...,0.119518,0.197589,0.113135,0.136490,0.162921,0.099021,0.086653,0.158176,0.166968,0.048967


In [92]:
%%time
maxs = emb3.max(axis=0)
mins = emb3.min(axis=0)
mins, maxs = da.compute(mins, maxs)
pd.DataFrame([mins, maxs], index=['min', 'max'], columns=es)

CPU times: total: 2.89 s
Wall time: 1.65 s


,e1,e2,e3,e4,e5,e6,e7,e8,e9,e10,...,e303,e304,e305,e306,e307,e308,e309,e310,e311,e312
min,-0.132803,-0.149056,-0.094468,-0.191697,-0.114229,-0.114341,-0.096039,-0.115178,-0.157275,-0.116715,...,-0.103254,-0.122285,-0.149789,-0.127703,-0.094802,-0.119690,-0.141425,-0.123732,-0.081543,-0.227348
max,0.135038,0.076125,0.157854,0.030987,0.101192,0.111774,0.147497,0.173821,0.099808,0.115573,...,0.119518,0.197589,0.113135,0.136490,0.162921,0.099021,0.086653,0.158176,0.166968,0.048967


<p class="task" id="4"></p>

4\. Датасет `embeddings` представляет собой набор 312-мерных векторов $x_i, i=0, 1, ... M-1$ Найдите вектор $x \ne x_{256}$ из набора данных, ближайший к вектору $x_{256}$ в смысле метрики $L_1$. Выведите на экран первые 10 координат вектора $x$.

$$d_1(\textbf{x},\textbf{y})=\sum_{k=1}^{n}{|x_i - y_i|}, \textbf{x}, \textbf{y} \in \mathbb{R}^n$$

In [4]:
#############
M, N = emb.shape[0], emb.shape[1]
r = M
c = N 
emb3 = da.asarray(emb, chunks=(r, c))

In [22]:
emb3_ = da.delete(emb3, 256, 0)
emb3_256 = emb3[256]
d = da.sum(da.abs(emb3_ - emb3_256), axis=1)
ind = da.argmin(d) 
ans = emb3_[ind][:10]

In [29]:
da.compute(emb3_256, ind, ans)[:10]

(1132464,
 array([ 0.0331987 , -0.03648246,  0.06629294, -0.0850755 , -0.04708353,
         0.00130241,  0.00259956,  0.01916818, -0.00985817, -0.04410348],
       dtype=float32))

<p class="task" id="5"></p>

5\. Рецепты разбиты на 4 группы. Загрузите маску для разбиения на группы из датасета `mask` из файла `recipe_embeddings.h5` в виде `dask.array`. Для каждой группы посчитайте и выведите на экран максимальное значение нормы $\ell_1$ векторов рецептов, принадлежащих к этой группе. 

Подсказка: закодируйте маску принадлежности к группе при помощи метода кодирования one-hot encoding и воспользуйтесь механизмом распространения.

$$\ell_1: ||\textbf{x}||_1=\sum_{k=1}^{n}{|x_k|}, \textbf{x} \in \mathbb{R}^n$$

In [30]:
f = h5py.File("recipe_embeddings.h5")
mask = da.array(f["mask"])
mask

dask.array<array, shape=(1200000,), dtype=int64, chunksize=(1200000,), chunktype=numpy.ndarray>

In [61]:
f.close()

In [13]:
mask[:20].compute()

array([0, 0, 1, 0, 2, 0, 0, 2, 2, 0, 1, 1, 0, 0, 1, 0, 0, 0, 2, 1],
      dtype=int64)

In [35]:
one_hot_mask = da.eye(4)[mask]
abss = da.abs(emb3)
norms = da.sum(abss, axis=1)

In [33]:
one_hot_mask.shape

(1200000, 4)

In [37]:
len(norms)

1200000

In [45]:
maksis = da.max(one_hot_mask * norms[:,None], axis=0)
maksis.compute()

array([13.31967735, 13.32409477, 13.31525993, 13.31915665])

<p class="task" id="6"></p>

6\. Работая с исходным файлом в формате `hdf`, реализуйте алгоритм подсчета среднего вектора датасета в блочной форме.

Блочный алгоритм вычислений состоит из двух частей:
1. Загрузка фрагмента за фрагментом данных и проведение вычислений над этим фрагментом
2. Агрегация результатов вычислений на различных фрагментах для получения результата на уровне всего набора данных

Важно: при работе с `hdf` в память загружаются не все элементы, а только те, которые запрашиваются в данный момент. При работе с `hdf` вы можете работать с массивами `numpy.array`. Для итерации по сегментам файла допускается использование циклов.

In [41]:
%%time
with h5py.File('recipe_embeddings.h5', 'r') as f:
    shape = f['embeddings'].shape
    size = 500
    dataset = f['embeddings']
    mean_vecs = np.array([])
    p = 0 

    for i in range(0, shape[0], size):
        p += 1
        j = min(i + size, shape[0])
        embedding = dataset[i:j]
        mean_vecs = np.append(
            mean_vecs, 
            np.sum(embedding, axis=1) / shape[1]
                             )
mean_vecs[:10]

CPU times: total: 8.41 s
Wall time: 9.57 s


array([0.00246155, 0.00246702, 0.00231133, 0.00252983, 0.00257839,
       0.00225921, 0.00245021, 0.00208791, 0.00230142, 0.00247999])

In [40]:
# проверка
len(mean_vecs)

1200000

<p class="task" id="7"></p>

7\. Решите задачу 6, распараллелив вычисления при помощи `ThreadPool`. Сравните время и результаты решения работы вашего алгоритма с реализацией поиска среднего вектора из `dask`. 

In [3]:
%%file cmm.py
import numpy as np
def count_mean(arr):
    return np.mean(arr, axis=1)

Overwriting cmm.py


In [4]:
from cmm import count_mean

In [14]:
from multiprocessing.pool import ThreadPool

In [16]:
%%time
with h5py.File('recipe_embeddings.h5', 'r') as f:
    shape = f['embeddings'].shape
    size = 500
    dataset = f['embeddings']
    mean_vecs = np.array([])

    for i in range(0, shape[0], size):
        j = min(i + size, shape[0])
        embedding = dataset[i:j]
        arrs = np.split(embedding, 100)
        with ThreadPool(processes=100) as pool:
            res = pool.map(count_mean, arrs)
        mean_vecs = np.append(mean_vecs, res)
mean_vecs[:10]

CPU times: total: 14.5 s
Wall time: 56.1 s


array([0.00246155, 0.00246702, 0.00231133, 0.00252983, 0.00257839,
       0.00225921, 0.00245021, 0.00208791, 0.00230142, 0.00247999])